# Cube Stacking with RL

This notebook trains a robot to stack one cube on top of another using reinforcement learning (SAC algorithm).

**Goal**: Learn to stack cube1 on top of cube2, minimizing vertical distance and maximizing horizontal alignment.

In [ ]:
import os
from pathlib import Path
import numpy as np

from applied_planning.envs import Lite6CubeStackingEnv

BASE_PATH = "/Users/braeden/Development/applied-planning/"
model_path = BASE_PATH + "src/applied_planning/sim/assets/ufactory_lite6/lite6_gripper_narrow.xml"

print("Creating cube stacking environment...")
env = Lite6CubeStackingEnv(
    model_path=str(model_path),
    render_mode=None,               # Set to "human" to watch training (will be slow!)
    max_steps=300,                  # More steps for complex stacking task
    success_threshold=0.08,         # 8cm vertical alignment
    horizontal_threshold=0.05,      # 5cm horizontal alignment
    reward_scale=10.0,
    cube_placement_radius=0.3,
    ee_site_name="end_effector",
    collision_penalty=100.0,
    terminate_on_collision=False
)

print(f"Observation space: {env.observation_space}")    # 16D: cube1(3) + cube2(3) + joints(6) + ee(3) + gripper(1)
print(f"Action space: {env.action_space}")              # 7D: joints(6) + gripper(1)

Creating cube stacking environment...
✓ Added 2 cubes to scene within 0.3m radius
IK solver using site: end_effector
Offscreen renderer initialized (notebook-friendly mode)
Observation space: Box(-inf, inf, (16,), float32)
Action space: Box(-1.0, 1.0, (7,), float32)


In [ ]:
# test env w/ random actions
obs, info = env.reset(seed=42)
print(f"Initial observation shape: {obs.shape}")
print(f"Cube 1 position: {info['cube1_pos']}")
print(f"Cube 2 position: {info['cube2_pos']}")
print(f"Vertical distance: {info['vertical_distance']:.4f} m")
print(f"Horizontal distance: {info['horizontal_distance']:.4f} m")

print("\nRunning random actions...")
for step in range(10):
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    print(f"Step {step+1}: v_dist={info['vertical_distance']:.4f} m, "
          f"h_dist={info['horizontal_distance']:.4f} m, "
          f"reward={reward:.2f}, collision={info['has_collision']}")
    if terminated or truncated:
        print(f"  Episode ended: success={info['is_success']}")
        break

Initial observation shape: (16,)
Cube 1 position: [-0.20456831  0.20575395  0.03      ]
Cube 2 position: [-0.02480644 -0.21832696  0.03      ]
Vertical distance: 0.0000 m
Horizontal distance: 0.4606 m

Running random actions...
Step 1: v_dist=0.0000 m, h_dist=0.4606 m, reward=-100.00, collision=True
Step 2: v_dist=0.0000 m, h_dist=0.4606 m, reward=0.00, collision=False
Step 3: v_dist=0.0000 m, h_dist=0.4606 m, reward=0.00, collision=False
Step 4: v_dist=0.0000 m, h_dist=0.4606 m, reward=-100.00, collision=True
Step 5: v_dist=0.0000 m, h_dist=0.4606 m, reward=0.00, collision=False
Step 6: v_dist=0.0000 m, h_dist=0.4606 m, reward=-100.00, collision=True
Step 7: v_dist=0.0000 m, h_dist=0.4606 m, reward=-100.00, collision=True
Step 8: v_dist=0.0000 m, h_dist=0.4606 m, reward=-100.00, collision=True
Step 9: v_dist=0.0000 m, h_dist=0.4606 m, reward=0.00, collision=False
Step 10: v_dist=0.0000 m, h_dist=0.4606 m, reward=-100.00, collision=True


## Train RL Agent

We use SAC (Soft Actor-Critic) which works well for continuous control tasks like cube stacking.

In [ ]:
from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import EvalCallback, CheckpointCallback

# make environment for stable-baselines3
def make_env():
    return Lite6CubeStackingEnv(
        model_path=str(model_path),
        render_mode=None,
        max_steps=300,
        success_threshold=0.08,
        horizontal_threshold=0.05,
        reward_scale=10.0,
        cube_placement_radius=0.3,
        ee_site_name="end_effector",
        collision_penalty=100.0,
        terminate_on_collision=False
    )

# create vectorized environments
train_env = DummyVecEnv([make_env])
eval_env = DummyVecEnv([make_env])

# setup callbacks
eval_callback = EvalCallback(
    eval_env,
    best_model_save_path="./logs/cube_stacking/best_model",
    log_path="./logs/cube_stacking/eval",
    eval_freq=5000,
    deterministic=True,
    render=False
)

checkpoint_callback = CheckpointCallback(
    save_freq=10000,
    save_path="./logs/cube_stacking/checkpoints",
    name_prefix="cube_stacking_model"
)

# soft actor critic model
print("\nCreating SAC model...")
model = SAC(
    "MlpPolicy",
    train_env,
    verbose=1,
    learning_rate=3e-4,
    buffer_size=100000,
    learning_starts=1000,
    batch_size=256,
    tau=0.005,
    gamma=0.99,
    train_freq=1,
    gradient_steps=1,
    tensorboard_log="./logs/cube_stacking/tensorboard"
)

print("\n" + "="*60)
print("Training Configuration:")
print("="*60)
print(f"Algorithm: SAC")
print(f"Task: Cube Stacking (2 cubes)")
print(f"Total timesteps: 200,000")
print(f"Eval frequency: 5,000 steps")
print(f"Checkpoint frequency: 10,000 steps")
print(f"Success criteria: vertical_dist < 0.08m, horizontal_dist < 0.05m")
print("="*60)

✓ Added 2 cubes to scene within 0.3m radius
IK solver using site: end_effector
Offscreen renderer initialized (notebook-friendly mode)
✓ Added 2 cubes to scene within 0.3m radius
IK solver using site: end_effector
Offscreen renderer initialized (notebook-friendly mode)

Creating SAC model...
Using cpu device

Training Configuration:
Algorithm: SAC
Task: Cube Stacking (2 cubes)
Total timesteps: 200,000
Eval frequency: 5,000 steps
Checkpoint frequency: 10,000 steps
Success criteria: vertical_dist < 0.08m, horizontal_dist < 0.05m


In [ ]:
print("\nStarting training...")
print("(This will take a while. Press Ctrl+C to stop early)\n")

try:
    model.learn(
        total_timesteps=200000,
        callback=[eval_callback, checkpoint_callback],
        progress_bar=True
    )

    # save final model
    model.save("./logs/cube_stacking/cube_stacking_final")
    print("\n✓ Training complete! Model saved to ./logs/cube_stacking/cube_stacking_final")

except KeyboardInterrupt:
    print("\n\nTraining interrupted by user")
    model.save("./logs/cube_stacking/cube_stacking_interrupted")
    print("Model saved to ./logs/cube_stacking/cube_stacking_interrupted")


Starting training...
(This will take a while. Press Ctrl+C to stop early)

Logging to ./logs/cube_stacking/tensorboard/SAC_1


Output()

---------------------------------
| rollout/           |          |
|    success_rate    | 0        |
| time/              |          |
|    episodes        | 4        |
|    fps             | 682      |
|    time_elapsed    | 1        |
|    total_timesteps | 1200     |
| train/             |          |
|    actor_loss      | 12.8     |
|    critic_loss     | 3.4e+03  |
|    ent_coef        | 0.942    |
|    ent_coef_loss   | -0.666   |
|    learning_rate   | 0.0003   |
|    n_updates       | 199      |
---------------------------------


---------------------------------
| rollout/           |          |
|    success_rate    | 0        |
| time/              |          |
|    episodes        | 8        |
|    fps             | 263      |
|    time_elapsed    | 9        |
|    total_timesteps | 2400     |
| train/             |          |
|    actor_loss      | -29.3    |
|    critic_loss     | 1.46e+03 |
|    ent_coef        | 0.762    |
|    ent_coef_loss   | -0.398   |
|    learning_rate   | 0.0003   |
|    n_updates       | 1399     |
---------------------------------


---------------------------------
| rollout/           |          |
|    success_rate    | 0        |
| time/              |          |
|    episodes        | 12       |
|    fps             | 228      |
|    time_elapsed    | 15       |
|    total_timesteps | 3600     |
| train/             |          |
|    actor_loss      | -212     |
|    critic_loss     | 964      |
|    ent_coef        | 0.682    |
|    ent_coef_loss   | -0.544   |
|    learning_rate   | 0.0003   |
|    n_updates       | 2599     |
---------------------------------


...

Success rate: 0.00%

----------------------------------
| eval/              |           |
|    mean_ep_length  | 300       |
|    mean_reward     | 7.31e+03  |
|    success_rate    | 0         |
| time/              |           |
|    total_timesteps | 150000    |
| train/             |           |
|    actor_loss      | -2.84e+03 |
|    critic_loss     | 6.67e+03  |
|    ent_coef        | 1.01      |
|    ent_coef_loss   | -0.00214  |
|    learning_rate   | 0.0003    |
|    n_updates       | 148999    |
----------------------------------


----------------------------------
| rollout/           |           |
|    success_rate    | 0.02      |
| time/              |           |
|    episodes        | 512       |
|    fps             | 173       |
|    time_elapsed    | 870       |
|    total_timesteps | 150611    |
| train/             |           |
|    actor_loss      | -3.04e+03 |
|    critic_loss     | 2.6e+03   |
|    ent_coef        | 1.02      |
|    ent_coef_loss   | 0.0027    |
|    learning_rate   | 0.0003    |
|    n_updates       | 149610    |
----------------------------------


----------------------------------
| rollout/           |           |
|    success_rate    | 0.02      |
| time/              |           |
|    episodes        | 516       |
|    fps             | 172       |
|    time_elapsed    | 877       |
|    total_timesteps | 151811    |
| train/             |           |
|    actor_loss      | -2.86e+03 |
|    critic_loss     | 4.79e+03  |
|    ent_coef        | 1.02      |
|    ent_coef_loss   | 0.00854   |
|    learning_rate   | 0.0003    |
|    n_updates       | 150810    |
----------------------------------


----------------------------------
| rollout/           |           |
|    success_rate    | 0.02      |
| time/              |           |
|    episodes        | 520       |
|    fps             | 172       |
|    time_elapsed    | 885       |
|    total_timesteps | 153011    |
| train/             |           |
|    actor_loss      | -2.86e+03 |
|    critic_loss     | 3.7e+03   |
|    ent_coef        | 0.983     |
|    ent_coef_loss   | 0.00631   |
|    learning_rate   | 0.0003    |
|    n_updates       | 152010    |
----------------------------------


## Test Trained Model

Visualize the trained agent attempting to stack cubes in MuJoCo.

In [ ]:
import time

print("="*60)
print("Testing Trained Model in MuJoCo Viewer")
print("="*60)

try:
    model = SAC.load("./logs/cube_stacking/best_model/best_model")
    print("✓ Loaded best model from evaluation")
except:
    model = SAC.load("./logs/cube_stacking/cube_stacking_final")
    print("✓ Loaded final model")

test_env = Lite6CubeStackingEnv(
    model_path=str(model_path),
    render_mode="human", # this will open MuJoCo viewer b/c we loading w human
    max_steps=300,
    success_threshold=0.08,
    horizontal_threshold=0.05,
    reward_scale=10.0,
    cube_placement_radius=0.3,
    ee_site_name="end_effector",
    collision_penalty=100.0,
    terminate_on_collision=False
)

print("\nRunning 5 test episodes...")
print("(MuJoCo viewer window should open)\n")

success_count = 0
total_rewards = []
collision_count = 0

for episode in range(5):
    obs, info = test_env.reset()
    print(f"\nEpisode {episode+1}:")
    print(f"  Cube 1 position: {info['cube1_pos']}")
    print(f"  Cube 2 position: {info['cube2_pos']}")
    print(f"  Initial vertical distance: {info['vertical_distance']:.4f} m")
    print(f"  Initial horizontal distance: {info['horizontal_distance']:.4f} m")
    
    episode_reward = 0
    episode_collisions = 0
    done = False
    
    for step in range(300):
        # get action from trained model
        action, _states = model.predict(obs, deterministic=True)
        
        # step environment
        obs, reward, terminated, truncated, info = test_env.step(action)
        episode_reward += reward
        
        if info['has_collision']:
            episode_collisions += 1
        
        # small delay so we can watch the movement
        time.sleep(0.02)
        
        if terminated or truncated:
            done = True
            break
    
    print(f"  Steps taken: {step+1}")
    print(f"  Final vertical distance: {info['vertical_distance']:.4f} m")
    print(f"  Final horizontal distance: {info['horizontal_distance']:.4f} m")
    print(f"  Success: {info['is_success']}")
    print(f"  Collisions: {episode_collisions}")
    print(f"  Total reward: {episode_reward:.2f}")
    
    if info['is_success']:
        success_count += 1
    collision_count += episode_collisions
    total_rewards.append(episode_reward)

test_env.close()

print("\n" + "="*60)
print("Test Summary:")
print("="*60)
print(f"Success rate: {success_count}/5 ({success_count/5*100:.1f}%)")
print(f"Average reward: {np.mean(total_rewards):.2f} ± {np.std(total_rewards):.2f}")
print(f"Total collisions: {collision_count}")
print("="*60)

In [ ]:
# Clean up
env.close()